In [ ]:
!pip install peft trl nltk tqdm groq

In [ ]:
from peft import get_peft_model, LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import PPOTrainer, PPOConfig
import torch
import random
from google.colab import userdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
import os
from google.cloud import storage
from safetensors import safe_open
import copy

### Simulated user profiles and preferences
we define a simulated spicy-loving user, who prefers spicy flavors

In [ ]:
user_profiles = {
    "spicy_lover": {
        "preferred_flavor": "spicy",
        "keywords": ["spicy"], #will store keywords that are related to spiciness
        "ingredients":["hot pepper", "hot sauce", "chili", "jalapeno"]  #will store ingredients that are related to spiciness
    }
}

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Now we enhance the user profiles to include keywords that are synonyms to their preferneces. The synonyms are determined by WordNet.

In [ ]:
def get_synonyms_and_related_terms(word):
    """
    Find synonyms and related terms using WordNet
    """
    related_terms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            related_terms.add(lemma.name())
    return related_terms

def enhance_user_profile(user_profile):
    """
    Extend the keywords and ingredients in the user_profile with synonyms
    """
    extended_profile = {
        "keywords": set(user_profile["keywords"]),
        "ingredients": set(user_profile["ingredients"])
    }

    for keyword in user_profile["keywords"]:
        # if the keyword has multiple words, join them with _ for nltk recognition
        keyword_underscored = keyword.replace(" ", "_") if " " in keyword else keyword

        related_keywords = get_synonyms_and_related_terms(keyword_underscored)
        # join the _ in the related word returned by nltk if the related word has it
        related_keywords_clean = [rk.replace("_", " ") if "_" in rk else rk for rk in related_keywords]

        extended_profile["keywords"].update(related_keywords_clean)

    # Enhance ingredients with related terms
    for ingredient in user_profile["ingredients"]:
        # if the ingredient has multiple words, join them with _ for nltk recognition
        ingredient_underscored = ingredient.replace(" ", "_") if " " in ingredient else ingredient

        related_ingredients = get_synonyms_and_related_terms(ingredient_underscored)
        # join the _ in the related word returned by nltk if the related word has it
        related_ingredients_clean = [ri.replace("_", " ") if "_" in ri else ri for ri in related_ingredients]

        extended_profile["ingredients"].update(related_ingredients_clean)

    extended_profile["keywords"] = list(extended_profile["keywords"])
    extended_profile["ingredients"] = list(extended_profile["ingredients"])

    return extended_profile

for user_type, user_profile in user_profiles.items():
    enhanced_profile = enhance_user_profile(user_profile)
    user_profile["extended_keywords"] = enhanced_profile["keywords"]
    user_profile["extended_ingredients"] = enhanced_profile["ingredients"]

### RLAIF

Since we do not have human feedback data, inspired by the work on RLAIF, we use the off-the-shell `llama-3.1-8b-instant` model (called through the GroqAPI) for preference labeling. The model decides from two generated recipes which one better suits the user's taste profile, since DPO instead of PPO will be implemented.

In [ ]:
from groq import Groq
import requests
groq_api_key = userdata.get('GROQ_API_KEY')
model = 'llama-3.1-8b-instant'
client = Groq(
      api_key=groq_api_key
  )

def get_ai_preference(generated_1, generated_2, user_type, user_profile):
    """
    RLAIF - an off-the-shelf LLM labels which recipe aligns better with the user's taste profile.
    """

    prompt = f"""
    You are an expert chef who understands user's taste profile well.
    A {user_type} user prefers recipes with the following characteristics:
    - Keywords: {', '.join(user_profile['extended_keywords'])}
    - Ingredients: {', '.join(user_profile['extended_ingredients'])}

    Based on the user's preferences, which of the two recipes aligns with the taste profile of the user better? Respond with "Recipe 1" or "Recipe 2" only, no additional tokens.

    1. Recipe 1: {generated_1}
    2. Recipe 2: {generated_2}
    """

    chat_completion = client.chat.completions.create(
        messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        model='llama-3.1-8b-instant'
    )
    choice_response = chat_completion.choices[0].message.content
    print(choice_response)

    # Determine the preferred recipe
    if "Recipe 1" in choice_response or "recipe 1" in choice_response:
        return generated_1
    elif "Recipe 2" in choice_response or "recipe 2" in choice_response:
        return generated_2
    else:
        return choice_response

In [ ]:
for user_type, user_profile in user_profiles.items():
    # simple generated recipes to demonstrate AIF performance
    generated_recipe_1 = "chili with jalapeno and hot sauce."
    generated_recipe_2 = "tomato soup recipe with a touch of basil."
    preferred_recipe = get_ai_preference(generated_recipe_1, generated_recipe_2, user_type, user_profile)
    print("Preferred Recipe:", preferred_recipe)

Recipe 1
Preferred Recipe: chili with jalapeno and hot sauce.


### Applying LoRA to finetuned opt125m model

We load the finetuned opt125m model from the `model.safetensors` file stored in our GCP bucket. We then apply LoRA to the model for efficient DPO training.

**Warning** You must upload the `recipe.json` file to the Colab runtime to properly load the finetuned model.

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'recipe.json'

# Initialize Google Cloud Storage client
client = storage.Client()

# Load the .safetensors file from GCP bucket
bucket_name = 'recipe-dataset'
file_name = "finetuned_model/model.safetensors"

bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

local_file_path = "model.safetensors"
blob.download_to_filename(local_file_path)
original_model_name = "facebook/opt-125m"

finetuned_tokenizer = AutoTokenizer.from_pretrained(original_model_name)
finetuned_model = AutoModelForCausalLM.from_pretrained(original_model_name)
state_dict = {}
with safe_open(local_file_path, framework="pt", device="cpu") as f:
    for key in f.keys():
        state_dict[key] = f.get_tensor(key)
finetuned_model.load_state_dict(state_dict, strict=False)
finetuned_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [ ]:
dpo_model = copy.deepcopy(finetuned_model)
dpo_tokenizer = AutoTokenizer.from_pretrained(original_model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dpo_model.to(device)

# LoRA
lora_config = LoraConfig(
    r=4,  # low-rank dimension
    lora_alpha=32,  # scaling factor
    lora_dropout=0.1,  # dropout (improve regularization)
    task_type="CAUSAL_LM"
)

# apply LoRA to the model
dpo_model = get_peft_model(dpo_model, lora_config)
dpo_model.print_trainable_parameters()

trainable params: 147,456 || all params: 125,386,752 || trainable%: 0.1176


In [ ]:
# define the optimizer only for trainable parameters
optimizer = torch.optim.AdamW(dpo_model.parameters(), lr=1e-5)

### Training loop with DPO

In [ ]:
def generate_recipe(model, tokenizer, prompt, preference=None):
    """
    Generates a recipe with a specified flavor; for DPO pairs
    """
    if preference:
        prompt += "The recipe should have a " + preference + " flavor"

    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
            inputs.input_ids,
            max_length=350,
            num_return_sequences=1,
            #attention_mask=inputs["attention_mask"],
            #pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.75,
            repetition_penalty=1.2
        )

    full_output = tokenizer.decode(output[0], skip_special_tokens=True)
    recipe_text = full_output[len(prompt):].strip()
    return recipe_text

In [ ]:
def pad_to_same_length(tensor_a, tensor_b, pad_value=0):
    """
    Pad tensors to the same length to facilitate DPO loss calculation
    """
    max_length = max(tensor_a.size(1), tensor_b.size(1))
    tensor_a_padded = F.pad(tensor_a, (0, max_length - tensor_a.size(1)), value=pad_value)
    tensor_b_padded = F.pad(tensor_b, (0, max_length - tensor_b.size(1)), value=pad_value)
    return tensor_a_padded, tensor_b_padded

In [ ]:
import torch.nn.functional as F
from tqdm import tqdm


num_epochs = 3
num_comparisons_per_epoch = 10  # number of DPO comparisons per epoch

for epoch in range(num_epochs):
    epoch_dpo_loss = 0  # total DPO loss for the epoch

    with tqdm(total=num_comparisons_per_epoch * len(user_profiles), desc=f"Epoch {epoch + 1}") as pbar:
        for _ in range(num_comparisons_per_epoch):
          for user_type, user_profile in user_profiles.items():
              prompt = "Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions."

              # generate two different recipes
              generated_spicy = generate_recipe(dpo_model,dpo_tokenizer, prompt, "spicy")
              generated_light = generate_recipe(dpo_model,dpo_tokenizer,prompt, "sweet")

              # use RLAIF to determine which recipe the user prefers
              preferred_output = get_ai_preference(generated_spicy, generated_light, user_type, user_profile)
              non_preferred_output = generated_light if preferred_output == generated_spicy else generated_spicy
              preferred_input_ids = dpo_tokenizer(preferred_output, return_tensors="pt").input_ids.to(dpo_model.device)
              non_preferred_input_ids = dpo_tokenizer(non_preferred_output, return_tensors="pt").input_ids.to(dpo_model.device)

              # get log probabilities for both outputs to compute gradients
              preferred_output_logits = dpo_model(preferred_input_ids).logits
              non_preferred_output_logits = dpo_model(non_preferred_input_ids).logits
              preferred_log_probs = preferred_output_logits.log_softmax(dim=-1)
              non_preferred_log_probs = non_preferred_output_logits.log_softmax(dim=-1)

              # ensure size of tensors (to avoid error)
              seq_len = min(preferred_log_probs.size(1), non_preferred_log_probs.size(1))
              preferred_log_probs = preferred_log_probs[:, :seq_len, :]
              non_preferred_log_probs = non_preferred_log_probs[:, :seq_len, :]

              # check vocab size of tensors (to avoid error)
              vocab_size = min(preferred_log_probs.size(-1), non_preferred_log_probs.size(-1))
              preferred_log_probs = preferred_log_probs[:, :, :vocab_size]
              non_preferred_log_probs = non_preferred_log_probs[:, :, :vocab_size]

              # calculate DPO loss
              dpo_loss = torch.mean(preferred_log_probs - non_preferred_log_probs)

              # update model weights
              optimizer.zero_grad()
              (-dpo_loss).backward()  # Negate to maximize the preference
              optimizer.step()

              epoch_dpo_loss += dpo_loss.item()

              pbar.update(1)
              print(f"++++++++ User Type: {user_type} ++++++++")
              print(f"++++++++ Prompt: {prompt} ++++++++")
              print(f"++++++++ Preferred Recipe: {preferred_output} ++++++++")
              print(f"++++++++ Non-Preferred Recipe: {non_preferred_output} ++++++++")
              print(f"++++++++ DPO Loss: {dpo_loss.item()} ++++++++")
              print("-" * 80)

    # average DPO loss for the epoch
    avg_epoch_loss = epoch_dpo_loss / (num_comparisons_per_epoch * len(user_profiles))
    print(f"******* Epoch {epoch + 1} completed with average DPO loss: {avg_epoch_loss:.4f} *******")
    print("=" * 100)


Epoch 1:   0%|          | 0/10 [00:00<?, ?it/s]

Recipe 2


Epoch 1:  10%|█         | 1/10 [00:10<01:34, 10.49s/it]

++++++++ User Type: spicy_lover ++++++++
++++++++ Prompt: Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions. ++++++++
++++++++ Preferred Recipe: and is recommended for those who like spicy meatballs but don't want to cook them in the microwave !Ingredients:
tomato
beef

Instructions:
1. mix all ingredients together well
2. heat over medium high heat , stirring occasionally until thickened , about 15 minute intervals
3. serve hot or cold
4. enjoy !
5. you can also use this as a side dish
6. it's a great substitute for rice , noodles , etc
7. i love the texture of this as well
8. i usually add more water if i'm using too much liquid , because it makes the sauce thicker and has less fat on it
9. i've never used this before , so i just put it in my fridge till i needed it
10. when i first started out , 

Epoch 1:  20%|██        | 2/10 [00:18<01:13,  9.15s/it]

...

Recipe 2
++++++++ User Type: spicy_lover ++++++++
++++++++ Prompt: Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions. ++++++++
++++++++ Preferred Recipe: and is recommended for those who want to add some meat to their meal but don't necessarily like it without it)
1. combine all ingredients in crock pot , cover and cook on high for 30 - 40 minutes or until desired doneness
2. serve immediately or freeze up to 1 month
3. this freezes very well and makes a great dipping sauce !
4. can also be made ahead and refrigerated at room temperature so you can keep them warm during your meal
5. i've also used this as a dip and it's been great ! :)
6. i love to put this into a sandwich bag and toss with my favorite toppings such as cheese , lettuce , tomatoes , onion etc
7. you get the idea ?
8. if you're not fa

Epoch 3:  90%|█████████ | 9/10 [01:17<00:08,  8.59s/it]

Recipe 2
++++++++ User Type: spicy_lover ++++++++
++++++++ Prompt: Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions. ++++++++
++++++++ Preferred Recipe: profile and detailed, step-by-step cooking instructions.Ingredients:
tomato
beef

Instructions:
1. combine all ingredients in a large bowl , mix well
2. refrigerate for at least 2 hours before serving
3. serve over your favorite pasta or rice
4. it is also great to serve on the side !
5. i love this dish as a dip , but if you want to add more meat / veggies , add some fat / water / salt to taste
6. just a little bit of both ! :)
7. i like to use my steaks , but it's not necessary -- they are very tender !
8. enjoy ! :)
9. we sometimes add some salt and pepper , but it doesn't make it any less delicious !
10. i've never tried adding anything else un

Epoch 3: 100%|██████████| 10/10 [01:27<00:00,  8.70s/it]

Recipe 2.
++++++++ User Type: spicy_lover ++++++++
++++++++ Prompt: Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions. ++++++++
++++++++ Preferred Recipe: and is recommended for those who like it sweeter but don't want to burn their meat.Ingredients:
tomato
beef

Instructions:
1. in a large bowl , mix together all ingredients except tomatoes
2. chill until ready to serve
3. this can also be made ahead and served warm or cold , depending on your preference
4. i usually make this at home and add some diced onions , if you prefer them soft
5. this will keep well in the refrigerator for up to 4 weeks
6. when ready to serve , toss lightly with some chopped tomato puree and serve ! :)
7. enjoy ! :)
8. my favorite is probably the cheddar cheese blend , as it's just so much better than most cheeses
9. great

### Compare original vs. RLHF model

In [ ]:
for user_type, user_profile in user_profiles.items():
    prompt = "Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions."
    print(user_type)
    original_output = generate_recipe(finetuned_model, finetuned_tokenizer, prompt)
    updated_output = generate_recipe(dpo_model, dpo_tokenizer, prompt)

    # Display outputs
    print(f"*****Prompt: {prompt}*****")
    print(f"*****Original Model Output: {original_output}*****")
    print(f"*****DPO Model Output: {updated_output}*****")
    print("-" * 50)

spicy_lover
*****Prompt: Please write a low-sodium meal recipe that takes approximately 55 minutes and includes the following ingredients: tomato, beef. The recipe should be formatted with a clear list of ingredients and detailed, step-by-step cooking instructions.*****
*****Original Model Output: Ingredients:
tomato
beef

Instructions:
1. cook meat according to package directions , drain and rinse meat well
2. heat oil in skillet over medium high heat until browned on both sides
3. add beef and sautee until no longer pink , about 5 minutes
4. stir in remaining ingredients
5. cover and cook for 30 minutes more or until meat is tender but not browned
6. serve hot over rice or noodles
7. can also be served as a dip with fresh mint leaves , a slice of lemon or lime slice , etc
8. enjoy ! :)
9. yum !
10. you may substitute other veggies such as spinach , broccoli , cauliflower , carrots , chicken stock , ground beef , onion , celery , etc
11. this also makes great dip , so try it !
12. it'